In [ ]:
#Scraping all financial data available from Oceans14 website creating dataframes with 'pandas' and ploting data with 'matplotlib', written by Capuccino.
#packages
!pip install selenium
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
!pip install pandas_datareader
import pandas_datareader.data as web
from pandas.api.types import CategoricalDtype
import os
from collections import defaultdict
#login
email = input('Email: ')
password = input('Senha: ')
#sector and tickers
s = input('Setor (separado por "/"): ')
tickers = input('Tickers (separado por ",": ')
w = tickers.replace(" ",",")
z = list(map(str,w.split(',')))
#getting correct urls for each ticker
for i in z:
    a=i
    print(a)
#login
    ticker = a
    loginp = 'https://www.oceans14.com.br/usuarios/frmLogin.aspx'
    driver = webdriver.Firefox()
    driver.get(loginp)
    time.sleep(2)
    driver.find_element(By.CSS_SELECTOR, "#ctl00_conteudoPrincipal_txtLogin").send_keys(email)
    driver.find_element(By.CSS_SELECTOR, "#ctl00_conteudoPrincipal_txtSenha").send_keys(password)
    time.sleep(2)
    buttom = driver.find_element(By.CSS_SELECTOR, "#ctl00_conteudoPrincipal_cmdEntrar")
    webdriver.ActionChains(driver).click(buttom).perform()
#inicio
    time.sleep(2)
    lupa = driver.find_element(By.CSS_SELECTOR, ".buscaTopoDesktop")
    webdriver.ActionChains(driver).click(lupa).perform()
    driver.find_element(By.CSS_SELECTOR, "#txtTopoPesquisaDesktop").send_keys(ticker)
    time.sleep(2)
    driver.find_element(By.CSS_SELECTOR, "#txtTopoPesquisaDesktop").send_keys(Keys.RETURN)
    time.sleep(2)
    geturl = driver.current_url
    if "balanco" in geturl:
        if driver.find_elements(By.CSS_SELECTOR, "#bsPainelLucratividade > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div"):
            res = driver.find_element(By.CSS_SELECTOR, "#bsPainelLucratividade > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1)").text
            ind = driver.find_element(By.CSS_SELECTOR, "#abaIndicadores01").text
            div = driver.find_element(By.CSS_SELECTOR, "#bsPainelDivida > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1)").text
            flx = driver.find_element(By.CSS_SELECTOR, "#bsPainelFluxoCaixa > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1)").text
            bal = driver.find_element(By.CSS_SELECTOR, "#bsBalancoPatrimonial > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1)").text
            driver.quit()
#add label for first column and remove symbols from variables
            resz = "Datas\n"+res.replace(".","").replace(" ","\n").replace("%","").replace(",",".")
            indz = "Datas\n"+ind.replace(".","").replace(" ","\n").replace("%","").replace(",",".")
            divz = "Datas\n"+div.replace(".","").replace(" ","\n").replace("%","").replace(",",".")
            flxz = "Datas\n"+flx.replace(".","").replace(" ","\n").replace("%","").replace(",",".")
            balz = "Datas\n"+bal.replace(".","").replace(" ","\n").replace("%","").replace(",",".")
            reszs = resz.splitlines()
            indzs = indz.splitlines()
            divzs = divz.splitlines()
            flxzs = flxz.splitlines()
            balzs = balz.splitlines()
#define headers for data
#resultados
            reszsr = defaultdict(list)
            key = None
            reset_key = True
            for item in reszs:
                if str(item).isdigit() or item.startswith('-') or item.endswith('%') or item.find(".")>0:
                    reszsr[key].append(item)
                    reset_key = True
                elif reset_key:
                    key = item
                    reset_key = False
                else:
                    key += f' {item}'
#indicadores
            indzsr = defaultdict(list)
            key = None
            reset_key = True
            for item in indzs:
                if str(item).isdigit() or item.startswith('-') or item.endswith('%') or item.find(".")>0:
                    indzsr[key].append(item)
                    reset_key = True
                elif reset_key:
                    key = item
                    reset_key = False
                else:
                    key += f' {item}'
#dívidas
            divzsr = defaultdict(list)
            key = None
            reset_key = True
            for item in divzs:
                if str(item).isdigit() or item.startswith('-') or item.endswith('%') or item.find(".")>0:
                    divzsr[key].append(item)
                    reset_key = True
                elif reset_key:
                    key = item
                    reset_key = False
                else:
                    key += f' {item}'
#fluxo de caixa
            flxzsr = defaultdict(list)
            key = None
            reset_key = True
            for item in flxzs:
                if str(item).isdigit() or item.startswith('-') or item.endswith('%') or item.find(".")>0:
                    flxzsr[key].append(item)
                    reset_key = True
                elif reset_key:
                    key = item
                    reset_key = False
                else:
                    key += f' {item}'
#balanço patrimonial
            balzsr = defaultdict(list)
            key = None
            reset_key = True
            for item in balzs:
                if str(item).isdigit() or item.startswith('-') or item.endswith('%') or item.find(".")>0:
                    balzsr[key].append(item)
                    reset_key = True
                elif reset_key:
                    key = item
                    reset_key = False
                else:
                    key += f' {item}'
#creating dataframes
            dfres = pd.DataFrame(reszsr).set_index(["Datas"]).replace('---','0')
            dfdiv = pd.DataFrame(divzsr).set_index(["Datas"]).replace('---','0')
            dfflx = pd.DataFrame(flxzsr).set_index(["Datas"]).replace('---','0')
            dfbal = pd.DataFrame(balzsr).set_index(["Datas"]).replace('---','0')
            dfind = pd.DataFrame(indzsr).set_index(["Datas"]).replace('---','0')  
#dataframes tò float
            dfresf = dfres.astype(float)
            dfdivf = dfdiv.astype(float)
            dfflxf = dfflx.astype(float)
            dfbalf = dfbal.astype(float)
            dfindf = dfind.astype(float)
#rename tabela indicadores
            dfindf.columns = ['LPA','P/L','VPA','P/VP','P/EBIT','P/EBITDA','P/Ativos','EBITDA','Marg EBITDA','PSR','ROE','ROA','ROIC','EV/EBIT','EV/EBITDA','Div Yield','Div Payout','Beta','Valorização','Negócios/dia','Volume Diário']
#creating final dataframe
            ca = dfresf.merge(dfdivf, left_index=True, right_index=True)
            cb = ca.merge(dfflxf, left_index=True, right_index=True)
            cc = cb.merge(dfbalf, left_index=True, right_index=True)
            complete = cc.merge(dfindf, left_index=True, right_index=True)
#managing plots
            fig1, f1_axes = plt.subplots(ncols=2, nrows=5, figsize=(30,20))
            fig1.suptitle (a, size=50)
            f1_axes[0, 0].plot(complete['Receita líquida'], lw=2, marker='^', markersize=15, label="Receita líquida")
            f1_axes[0, 0].plot(complete['Resultado bruto'], lw=2, marker='.', markersize=10, label="Resultado bruto")
            f1_axes[0, 0].plot(complete['Lucro líquido'], lw=2, marker='.', markersize=10, label="Lucro líquido")
            f1_axes[1, 0].plot(complete['Margem bruta'], lw=2, marker='.', markersize=10, label="Margem bruta")
            f1_axes[1, 0].plot(complete['Margem Oper'], lw=2, marker='.', markersize=10, label="Margem Oper")
            f1_axes[1, 0].plot(complete['Margem líquida'], lw=2, marker='^', markersize=15, label="Margem líquida")
            f1_axes[0, 1].plot(complete['Patrimônio líq'], lw=2, marker='^', markersize=15, label="Patrimônio líq")
            f1_axes[0, 1].plot(complete['Dívida bruta'], lw=2, marker='.', markersize=10, label="Dívida bruta")
            f1_axes[0, 1].plot(complete['Dívida líq'], lw=2, marker='.', markersize=10, label="Dívida líq")
            f1_axes[0, 1].plot(complete['Ativo Total'], lw=2, marker='.', markersize=10, label="Ativo Total")
            f1_axes[0, 1].plot(complete['Passivo Total'], lw=2, marker='.', markersize=10, label="Passivo Total")
            f1_axes[1, 1].plot(complete['FCL'], lw=2, marker='.', markersize=10, label="Fluxo de Caixa Livre")
            f1_axes[1, 1].plot(complete['FCO'], lw=2, marker='.', markersize=10, label="Fluxo de Caixa Operacional")
            f1_axes[1, 1].plot(complete['FCI'], lw=2, marker='.', markersize=10, label="Fluxo de Caixa de Investimentos")
            f1_axes[1, 1].plot(complete['FCF'], lw=2, marker='.', markersize=10, label="Fluxo de Caixa de Financiamento")
            f1_axes[1, 1].plot(complete['FCT'], lw=2, marker='^', markersize=15, label="Fluxo de Caixa Total")
            f1_axes[2, 0].plot(complete['Dívida bruta'], lw=2, marker='.', markersize=10, label="Dívida bruta")
            f1_axes[2, 0].plot(complete['Dívida líq'], lw=2, marker='^', markersize=15, label="Dívida líquida")
            f1_axes[2, 1].plot(complete['EBITDA'], lw=2, marker='.', markersize=10, label="EBITDA")
            f1_axes[3, 0].plot(complete['VPA'], lw=2, marker='.', markersize=10, label="VPA")
            f1_axes[3, 1].plot(complete['P/VP'], lw=2, marker='.', markersize=10, label="P/VP")
            f1_axes[3, 1].plot(complete['Beta'], lw=2, marker='.', markersize=10, label="Beta")
            f1_axes[3, 1].plot(complete['PSR'], lw=2, marker='.', markersize=10, label="Preço/Receita")
            f1_axes[4, 0].plot(complete['ROE'], lw=2, marker='.', markersize=10, label="ROE")
            f1_axes[4, 0].plot(complete['ROA'], lw=2, marker='.', markersize=10, label="ROA")
            f1_axes[4, 0].plot(complete['ROIC'], lw=2, marker='.', markersize=10, label="ROIC")
            f1_axes[4, 1].plot(complete['Valorização'], lw=2, marker='.', markersize=10, label="Valorização")
            f1_axes[0, 0].legend()
            f1_axes[0, 1].legend()
            f1_axes[1, 0].legend()
            f1_axes[1, 1].legend()
            f1_axes[2, 0].legend()
            f1_axes[2, 1].legend()
            f1_axes[3, 0].legend()
            f1_axes[3, 1].legend()
            f1_axes[4, 0].legend()
            f1_axes[4, 1].legend()
#save files
            if not os.path.exists("STOCKBR/"+s+"/"+a):
                os.makedirs("STOCKBR/"+s+"/"+a)
                plt.savefig("STOCKBR/"+s+"/"+a+"/"+a+"data.png")
                complete.to_excel(os.path.join("STOCKBR/"+s+"/"+a, a+".xlsx"),sheet_name=a)
                print("SUCCESS")
                driver.quit()
        else:
            print("MISSING TICKER")
            driver.quit()
    else:
        print("INVALID TICKER")
        driver.quit()
print("FINISHED")
driver.quit()